In [59]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

ArgumentError: ArgumentError: Package Turing not found in current path.
- Run `import Pkg; Pkg.add("Turing")` to install the Turing package.

In [32]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

UndefVarError: UndefVarError: `DataFrame` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [33]:
colnames(data_source)

UndefVarError: UndefVarError: `colnames` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [34]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi, dy)

UndefVarError: UndefVarError: `percentchange` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [35]:
CSV.write("usa_data_transformed.csv", data)

UndefVarError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [36]:
start = Date(1998,01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

UndefVarError: UndefVarError: `Date` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing Dates in the current active module Main

In [37]:
returns_summarystats(s_data, 1)

UndefVarError: UndefVarError: `returns_summarystats` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [38]:
returns_summarystats(s_data, freq)

UndefVarError: UndefVarError: `returns_summarystats` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [39]:
vmodel = VARModel(s_data)
model_summary(vmodel)

UndefVarError: UndefVarError: `VARModel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [40]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

UndefVarError: UndefVarError: `NormalWishartBVAR` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [41]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

UndefVarError: UndefVarError: `colnames` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [42]:
y_data = annualise(scenarios[1, :, 21:25],4)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

UndefVarError: UndefVarError: `annualise` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [43]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

UndefVarError: UndefVarError: `simulate` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [44]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [45]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


UndefVarError: UndefVarError: `print_percentiles` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [46]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

UndefVarError: UndefVarError: `s_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [47]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



UndefVarError: UndefVarError: `mu` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [48]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

UndefVarError: UndefVarError: `print_percentiles` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [49]:
returns_summarystats(data_source[:Yeld_10Y],1)

UndefVarError: UndefVarError: `returns_summarystats` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [50]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [51]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [52]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [53]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [54]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.1829501420163906
 -0.02774021741262327
  0.08058665402014775
  0.1876438889977062
  0.3427007624671199

In [55]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18413922684495396
 -0.027373339913322862
  0.0808094150943468
  0.18853023800657984
  0.34284237873092704

In [56]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

5-element Vector{Float64}:
 -0.25356624059034005
 -0.018194266686793498
  0.040347832023202204
  0.09897408495627744
  0.33721538836876624

In [57]:
r = rand(Normal(0,3),10_000)
display(skewness(r))
display(kurtosis(r))

-0.007937168395933782

-0.05553383435527426